In [53]:
# EDA
import pandas as pd
import pingouin as pg
import plotly.express as px
import plotly.figure_factory as ff  
import matplotlib.pyplot as plt

# Machine Learning
from sklearn.model_selection import cross_validate, StratifiedKFold, cross_val_predict, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

# Otimizacao de hiperparametros
import optuna

In [54]:
# Carregar Dataset
df_segmento = pd.read_csv('./datasets/segmento_clientes.csv')

### EDA

In [55]:
# Visualizar os dados
df_segmento.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao,segmento_de_cliente
0,Comércio,713109.95,12,Rio de Janeiro,6,1,Bronze
1,Comércio,790714.38,9,São Paulo,15,0,Bronze
2,Comércio,1197239.33,17,São Paulo,4,9,Silver
3,Indústria,449185.78,15,São Paulo,6,0,Starter
4,Agronegócio,1006373.16,15,São Paulo,15,8,Silver
5,Serviços,1629562.41,16,Rio de Janeiro,11,4,Silver
6,Serviços,771179.95,13,Vitória,0,1,Starter
7,Serviços,707837.61,16,São Paulo,10,6,Silver
8,Comércio,888983.66,17,Belo Horizonte,10,1,Bronze
9,Indústria,1098512.64,13,Rio de Janeiro,9,3,Bronze


In [56]:
# # Estrutuda do dataset
df_segmento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   atividade_economica     500 non-null    object 
 1   faturamento_mensal      500 non-null    float64
 2   numero_de_funcionarios  500 non-null    int64  
 3   localizacao             500 non-null    object 
 4   idade                   500 non-null    int64  
 5   inovacao                500 non-null    int64  
 6   segmento_de_cliente     500 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 27.5+ KB


In [57]:
# Valores possiveis - variaveis categoricas
df_segmento['atividade_economica'].unique()

array(['Comércio', 'Indústria', 'Agronegócio', 'Serviços'], dtype=object)

In [58]:
# Valores possiveis - variaveis categoricas
df_segmento['localizacao'].unique()

array(['Rio de Janeiro', 'São Paulo', 'Vitória', 'Belo Horizonte'],
      dtype=object)

In [59]:
# Valores possiveis - variaveis categoricas
df_segmento['segmento_de_cliente'].unique()

array(['Bronze', 'Silver', 'Starter', 'Gold'], dtype=object)

In [60]:
# Valores possiveis - variaveis categoricas
df_segmento['inovacao'].unique()

array([1, 0, 9, 8, 4, 6, 3, 7, 5, 2])

In [61]:
# Distribuicaoda variavel segmento de cliente (Target)
contagem_target = df_segmento.value_counts('segmento_de_cliente')
contagem_target

segmento_de_cliente
Silver     260
Bronze     202
Starter     22
Gold        16
Name: count, dtype: int64

In [62]:
# Criar uma lista ordenada do target
list_segmentos = ['Starter', 'Bronze', 'Silver', 'Gold']

In [63]:
# Distribuicao da variavel target - contagem
px.bar(contagem_target, color=contagem_target.index, category_orders={'segmento_de_cliente': list_segmentos})

In [64]:
# Distribuicao da variavel target - percentual
percentual_target = contagem_target / len(df_segmento) * 100
px.bar(percentual_target, color=percentual_target.index, category_orders={'segmento_de_cliente': list_segmentos})

In [65]:
# Distribuicao da variavel localizacao - percentual
percentual_localizacao = df_segmento['localizacao'].value_counts() / len(df_segmento) * 100
px.bar(percentual_localizacao, color=percentual_localizacao.index)

In [66]:
# Distribuicao da variavel atividade economica - percentual
percentual_atividade = df_segmento['atividade_economica'].value_counts() / len(df_segmento) * 100
px.bar(percentual_atividade, color=percentual_atividade.index)

In [67]:
# Distribuicao da variavel inovacao - percentual
percentual_inovacao = df_segmento['inovacao'].value_counts() / len(df_segmento) * 100
px.bar(percentual_inovacao, color=percentual_inovacao.index)